<a href="https://colab.research.google.com/github/Juhwan01/DeepDive/blob/main/KLUE_BERT_%EA%B0%9C%EC%B2%B4%EB%AA%85_%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이전과 똑같이 레거시 모드로
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
!pip install seqeval

## BIO 태그 체계 정리

### ✅ BIO란?
- B-XXX: 개체명 시작 (Begin)
- I-XXX: 개체명 내부 (Inside)
- O: 개체명 아님 (Outside)

### 📌 예시 문장
**문장:** 홍길동은 서울대학교를 졸업했다.

| 단어       | 태그    | 설명                         |
|------------|---------|------------------------------|
| 홍길동     | B-PER   | 사람 이름 시작               |
| 은         | O       | 개체명 아님                  |
| 서울대학교 | B-ORG   | 기관 이름 시작               |
| 를         | O       | 개체명 아님                  |
| 졸업했다   | O       | 개체명 아님                  |

### 🎯 여러 단어로 된 개체명 예시

**문장:** 대한 민국 정부는

| 단어   | 태그    | 설명               |
|--------|---------|--------------------|
| 대한   | B-ORG   | 기관 이름 시작     |
| 민국   | I-ORG   | 기관 이름 내부     |
| 정부   | I-ORG   | 기관 이름 내부     |

### 🏷️ 자주 쓰이는 개체명 태그

| 태그       | 의미           |
|------------|----------------|
| PER        | 사람 이름      |
| LOC        | 위치/장소      |
| ORG        | 기관/회사 이름 |
| DATE       | 날짜           |
| TIME       | 시간           |
| MONEY      | 금액           |
| PERCENT    | 백분율         |
| O          | 개체명 아님    |

### 🔄 다른 태그 체계 (참고용)
- BIO: 가장 일반적인 태그 체계
- BILOU: Begin, Inside, Last, Outside, Unit
- BIOES: Begin, Inside, Outside, End, Single


In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
# seqeval은 NER처럼 문장 내 단어마다 라벨이 붙는 시퀀스 라벨링 작업의 성능을 평가할 때 사용함
# 일반적인 정확도와 다르게, 토큰 단위가 아니라 엔티티 단위 평가도 가능함
# 예: 예측값과 실제값을 시퀀스 단위로 비교함
#y_true = [["B-PER", "I-PER", "O", "B-LOC"]]
#y_pred = [["B-PER", "I-PER", "O", "B-ORG"]]
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request

In [ ]:
import urllib.request

# 학습 데이터 다운로드
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/refs/heads/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv",
    filename="ner_train_data.csv"
)

# 테스트 데이터 다운로드
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/refs/heads/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv",
    filename="ner_test_data.csv"
)

# 라벨 정보 다운로드
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/refs/heads/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt",
    filename="ner_label.txt"
)


In [ ]:
# CSV 파일 로드
train_ner_df = pd.read_csv("ner_train_data.csv")[:30000]
test_ner_df = pd.read_csv("ner_test_data.csv")

In [ ]:
train_ner_df.head()

In [ ]:
test_ner_df.head()

In [ ]:
print('훈련용 샘플 개수 :', len(train_ner_df))
print('테스트용 샘플 개수 :', len(test_ner_df))

In [ ]:
# 학습 데이터의 'Sentence' 열에서 각 문장을 공백 기준으로 토큰화하여 리스트로 저장
# 예: "이순신 은 위인 이다" → ['이순신', '은', '위인', '이다']
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]

# 학습 데이터의 'Tag' 열에서 각 문장에 대한 개체명 태그를 공백 기준으로 분할하여 리스트로 저장
# 예: "PER-B O O O" → ['PER-B', 'O', 'O', 'O']
train_data_label = [label.split() for label in train_ner_df['Tag'].values]

# 테스트 데이터의 'Sentence' 열에서 각 문장을 공백 기준으로 토큰화하여 리스트로 저장
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]

# 테스트 데이터의 'Tag' 열에서 각 문장에 대한 개체명 태그를 공백 기준으로 분할하여 리스트로 저장
test_data_label = [label.split() for label in test_ner_df['Tag'].values]


In [ ]:
print(train_data_sentence[2])
print(train_data_label[2])

In [ ]:
# 이 데이터는 형태소 단위 x -> 어절(띄어쓰기) 단위로 개체명 인식
# "ner_label.txt" 파일을 읽어서 각 라벨을 리스트로 저장
# open() 함수로 파일을 읽기 모드('r')와 UTF-8 인코딩을 지정하여 연다
# 파일의 각 줄(line)을 읽어와 strip() 함수로 앞뒤 공백이나 줄바꿈 문자를 제거
# 이렇게 정제된 라벨들을 리스트에 담아 labels 변수에 저장
labels = [label.strip() for label in open("ner_label.txt", 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)


In [ ]:
# labels 리스트에 있는 개체명 태깅(tag)들을 인덱스와 매핑하는 딕셔너리 생성
# enumerate()는 index 같이 리턴

# tag_to_index: 태깅 이름(tag)을 키(key)로, 해당 태깅의 인덱스(index)를 값(value)으로 저장하는 딕셔너리
# 예를 들어, 'B-PER':0, 'I-PER':1 같은 형태로 태깅 이름을 숫자로 변환하는 용도
tag_to_index = {tag: index for index, tag in enumerate(labels)}
print(tag_to_index)

# index_to_tag: 인덱스를 키(key)로, 태깅 이름(tag)을 값(value)으로 저장하는 딕셔너리
# 숫자로 된 인덱스를 다시 원래 태깅 이름으로 변환할 때 사용합니다.
index_to_tag = {index: tag for index, tag in enumerate(labels)}
print(index_to_tag)

# 이 과정을 하는 이유:
# 모델 학습 시에는 태깅(label)을 숫자(index) 형태로 변환해야 컴퓨터가 이해하고 처리할 수 있음
# 반대로, 모델의 예측 결과인 숫자 인덱스를 다시 사람이 이해할 수 있는 태깅 이름으로 바꾸기 위해 index_to_tag가 필요함


In [ ]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :', tag_size)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
sent = train_data_sentence[1]
label = train_data_label[1]

print('문장 :',sent)
print('레이블 :',label)
# 매핑 딕셔너리에서 찾아서 정수 인코딩
print('레이블의 정수 인코딩 :', [tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

In [ ]:
tokens = []
# 예를 들어 '쿠마리'라는 단어가 BERT 토크나이저에 의해 ['쿠', '##마리']로 분리된 경우,
# 원래 단어 '쿠마리'에는 'B-PER'이라는 개체명 태그가 붙어 있었음.

# 문제는 이렇게 단어가 서브워드로 나뉘면, 레이블의 개수와 토큰의 개수가 일치하지 않게 되므로
# 모델에 넣기 위해서는 서브워드 수에 맞춰 레이블도 확장해야 함.

# 이때 일반적으로 사용하는 방식:
# - 첫 번째 서브워드(예: '쿠')에는 원래 레이블(B-PER 등)을 그대로 부여
# - 그 뒤에 따라오는 서브워드(예: '##마리')에는 레이블을 부여하지 않음 (== 무시됨)

# 이렇게 레이블을 부여하지 않는 위치에는 보통 '-100'을 넣음.
# - 이 값은 PyTorch나 TensorFlow 등에서 손실 계산 시 해당 위치를 무시하도록 처리됨.
for one_word in sent:
  # 각 단어 서브워드로 분리
  subword_tokens = tokenizer.tokenize(one_word)
  # extend()는 리스트 안의 요소들을 하나씩 추가해준다 <-> append()는 리스트 안에 리스트로 들어감
  tokens.extend(subword_tokens)

print('BERT 토크나이저 정룰 출력 :', tokens)
print('레이블 :', label)
print('레이블의 정수 인코딩 :', [tag_to_index[idx] for idx in label])
print('토큰의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

In [ ]:
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  # 라벨 ID 리스트에 추가
  labels_ids.extend(
      [tag_to_index[label_token]]          # => [1] : 첫 번째 subword에는 원래 라벨 그대로 사용
      + [-100] * (len(subword_tokens) - 1) # => [-100, -100] : 나머지 subword는 무시(-100) 처리
  )
print('토큰화 된 문장 :', tokens)
print('레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('패딩에 대한 정수 인코딩 :', labels_ids)
print('토큰의 길이 :', len(tokens))
print('레이블의 길이 :', len(labels_ids))



# 🧠 `convert_examples_to_features` 함수 전체 흐름 정리

이 함수는 NER 데이터셋(예: 단어/라벨 쌍 리스트)을 BERT 입력 형식에 맞게 **정제하고 변환하는 전처리 함수**입니다. 최종적으로는 BERT가 요구하는 4가지 입력을 생성합니다:

* `input_ids`
* `attention_mask`
* `token_type_ids`
* `data_labels` (NER용 라벨 시퀀스)

## 1️⃣ 특수 토큰 정보 가져오기

* BERT 모델은 문장 앞에 `[CLS]`, 끝에 `[SEP]` 같은 **특수 토큰**이 필요합니다.
* Hugging Face 토크나이저는 이런 특수 토큰들을 자동으로 제공합니다.
* 예를 들어:

  * `[CLS]`: 문장의 시작
  * `[SEP]`: 문장의 끝
  * `[PAD]`: 패딩 자리 채우기용
* 이 토큰들은 모델의 동작에 매우 중요하며, 시퀀스 분류나 문장쌍 관계 파악에도 사용됩니다.

## 2️⃣ 각 문장-라벨 쌍에 대해 반복

NER 데이터는 보통 다음과 같은 구조입니다:

```python
examples = [['나는', '학생이다'], ['너는', '의사다']]
labels = [['O', 'B-PER'], ['O', 'B-PROF']]
```

* 각 단어를 BERT 토크나이저로 **서브워드 수준까지 분해**합니다.
* 예: `'학생이다' → ['학생', '##이다']`
* 문제: 서브워드로 나뉘면 원래 단어의 라벨을 **복수의 토큰에 분배**해야 함

### 👉 라벨 처리 방법

* 첫 서브워드에는 **정상 라벨 부여**
* 나머지 서브워드에는 \*\*패딩용 라벨값 (-100)\*\*을 부여 (손실 계산에서 무시됨)

## 3️⃣ 너무 긴 문장은 잘라냄

* BERT는 입력 토큰 수가 **최대 512개**지만, 이 함수에서는 예를 들어 `max_seq_len = 64`로 제한한다고 가정
* `[CLS]`와 `[SEP]`을 포함해야 하므로, 실제 단어는 **64 - 2 = 62개까지만** 사용할 수 있음
* 이를 초과한 토큰은 뒤에서 잘라냅니다 (**truncation**)

## 4️⃣ `[CLS]`, `[SEP]` 특수 토큰 붙이기

* 앞에 `[CLS]`, 뒤에 `[SEP]`을 붙입니다
* 토큰뿐 아니라, 라벨 배열에도 같은 위치에 `-100`을 붙입니다
  이유: 이 특수 토큰들엔 라벨이 없으므로 손실 계산에서 제외해야 합니다

## 5️⃣ 정수 인코딩 + 어텐션 마스크 + 세그먼트 아이디

* BERT는 텍스트를 그대로 받지 않고, **정수 ID 배열**로 변환해야 합니다
* 토큰 배열을 `input_ids`로 바꾸고
  각 토큰 위치엔 1, 패딩엔 0을 두는 **attention\_mask**도 생성합니다
* `token_type_ids`는 문장쌍일 때 문장 구분용인데, 여기선 전부 0으로 설정합니다 (단일 문장이므로)

## 6️⃣ 패딩 추가

* 최대 길이보다 짧은 문장은 부족한 만큼 `[PAD]`로 채웁니다
* `input_ids`, `attention_mask`, `token_type_ids`, `data_labels` 모두 **동일한 길이로 맞춰야** 하므로,
  각각 적절한 패딩 값으로 맞춥니다:

  * 입력 ID → `[PAD]` 토큰의 인덱스
  * 마스크 → `0`
  * 세그먼트 ID → `0`
  * 레이블 → `-100` (무시용)

## 7️⃣ 정합성 체크 (디버깅 용도)

* 각 배열의 길이가 `max_seq_len`과 정확히 일치하는지 `assert`로 확인합니다
* 개발 중 잘못된 길이의 데이터가 들어오는 것을 방지합니다

## 8️⃣ 모든 결과 리스트에 누적

* 하나의 문장 처리가 끝나면, `input_ids`, `attention_mask`, `token_type_ids`, `data_labels` 리스트에 각각 추가합니다
* 리스트 안에 **한 문장에 대한 결과가 배열로 들어가는 구조**입니다

## 9️⃣ NumPy 배열로 변환

* 학습 전에 모든 리스트를 NumPy 배열로 변환합니다
* 텐서플로우 모델은 일반적으로 NumPy 또는 Tensor 형태의 데이터를 받기 때문입니다

## 🔚 최종 출력

함수는 다음을 반환합니다:

```python
(
  (input_ids, attention_mask, token_type_ids),  # BERT 입력
  data_labels                                   # NER 라벨
)
```

## ✅ 요약

| 단계 | 설명                                    |
| -- | ------------------------------------- |
| 1  | 단어 → 서브워드 토크나이징 및 라벨 확장               |
| 2  | `[CLS]`, `[SEP]` 붙이기                  |
| 3  | 시퀀스 길이 초과 시 자르기                       |
| 4  | 정수 인코딩, attention mask, segment id 생성 |
| 5  | 패딩 추가 및 길이 맞춤                         |
| 6  | 모든 항목 리스트에 누적                         |
| 7  | NumPy 배열로 변환 후 반환                     |




In [ ]:
def convert_examples_to_features(examples,labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0,
                                 pad_token_id_for_label=-100):
    # Hugging Face 토크나이저에서 특수 토큰 정보를 추출
    # 문장의 시작을 나타내는 특수 토큰 (예: '[CLS]')
    # BERT에서는 문장의 첫 토큰으로 항상 사용되며, 분류 작업에서 중요한 역할을 한다
    cls_token = tokenizer.cls_token

    # 문장의 끝 또는 문장 사이의 구분을 나타내는 특수 토큰 (예: '[SEP]')
    # BERT는 문장 하나일 경우 문장 끝에, 문장 두 개일 경우 중간과 끝에 [SEP]를 넣는다
    sep_token = tokenizer.sep_token

    # 패딩 토큰의 숫자 인덱스 (예: 0)
    # 입력 시퀀스를 동일한 길이로 맞추기 위해 사용되며, 손실 계산 시 무시됨
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [],[],[],[]

    for example, label in tqdm(zip(examples,labels), total = len(examples)):
        tokens = []
        labels_ids = []
        # 위에 해본것과 같이 토큰화 진행
        for one_word, label_token in zip(example,label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+[pad_token_id_for_label]*(len(subword_tokens)-1))
        # BERT의 최대 시퀀스 길이 제한: BERT는 보통 512 토큰까지만 처리 가능
        # 특수 토큰 공간 확보: [CLS]와 [SEP] 토큰을 위한 자리(2개) 필요
        special_tokens_count = 2
        # max_seq_len = 64로 설정했다면
        # 실제 토큰은 64 - 2 = 62개까지만 사용
        # 62개보다 긴 문장은 뒤쪽을 잘라냄 (truncation)
        # [CLS] + 토큰들(최대62개) + [SEP] 형태로 구성
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # sep 토큰 추가
        # 1. 토큰화 결과 끝 [SEP] 토큰 추가
        # 2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        # cls 토큰 추가
        # 1. 토큰화 결과 앞 [CLS] 토큰 추가
        # 2. 레이블의 맨 앞 부분에도 -100 추가
        tokens = [cls_token]+tokens
        labels_ids = [pad_token_id_for_label]+labels_ids

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)

        # 어텐션 마스크 생성
        attention_mask = [1] * len(input_id)

        # 정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(tokens)

        # 정수 인코딩, 어텐션 마스크 패딩 추가
        input_id = input_id + ([pad_token_id]*padding_count)
        attention_mask = attention_mask + ([0]*padding_count)

        # 세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment]*max_seq_len

        # 레이블 패딩 -> 여기서는 패딩 아이디 -100
        label = labels_ids + ([pad_token_id_for_label]*padding_count)

        # assert -> 디버깅 도구(앞 노트에서 설명함)
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        # 최종 결과를 List에 이어 붙인다 이 경우 append이기 때문에 [[]] 이런식으로 리스트 안에 리스트로 들어간다
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    # 앞의 노트에서 설명한 것처럼 np 배열 형태로 전환
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
X_train, y_train = convert_examples_to_features(train_data_sentence,
                                                train_data_label,max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence,
                                              test_data_label,max_seq_len=128, tokenizer=tokenizer)

In [ ]:
print('기존 원문 :', train_data_sentence[0])
print('기존 레이블 :', train_data_label[0])
print('-'*50)
print('토큰화 후 원문 :',tokenizer.convert_ids_to_tokens(X_train[0][0]))
# 트레인 데이터 라벨의 첫 번째 행의 값 = 배열
# 반복문 돌아서 하나하나 요소 체크하면서 -100 이면 [PAD] 아니면 매핑 딕셔너리에 맞는 토큰 출력
print('토큰화 후 레이블 :',['[PAD]' if index == -100 else index_to_tag[index] for index in y_train[0]])
print('-'*50)
print('정수 인코딩 결과',X_train[0][0])
print('정수 인코딩 레이블',y_train[0])

In [ ]:
print('세그먼트 인코딩 :', X_train[2][0])

In [ ]:
print('어텐션 마스크 :', X_train[1][0])

In [ ]:
# KoBERT를 이용한 개체명 인식(NER) 구현 - 다대다(Many-to-Many) 구조
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super().__init__()

        # 사전 학습된 KoBERT 모델 불러오기 (PyTorch 모델을 TF에서 사용)
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)

        # 출력층: 각 토큰에 대해 num_labels개의 클래스로 분류해야 하므로 Dense(num_labels) 사용
        # 입력: (batch_size, seq_len, hidden_size=768)
        # 출력: (batch_size, seq_len, num_labels)
        # 활성화 함수는 사용하지 않음 → softmax는 손실 함수에서 처리함 -> 나중에 아래 loss정의 부분에서 from_logits=True -> 내부적으로 softmax(logits)를 먼저 계산한 다음에 cross entropy를 계산함.
        self.classifier = tf.keras.layers.Dense(
            num_labels,
            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
            name='classifier'
        )

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs

        # BERT 모델의 출력: 모든 토큰에 대한 임베딩 벡터
        # shape: (batch_size, seq_len=128, hidden_size=768)
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # 개체명 인식은 문장이 아닌 각 토큰을 분류하는 작업이므로
        # BERT의 시퀀스 출력 전체(outputs[0])를 사용함
        all_output = outputs[0]  # shape: (batch_size, 128, 768)

        # 각 토큰의 벡터(768차원)를 Dense 레이어를 통해 13개 클래스 중 하나로 분류
        # 최종 출력: (batch_size, 128, 13)
        prediction = self.classifier(all_output)

        return prediction


In [ ]:
# tf.constant란?
# tf.constant는 TensorFlow에서 상수 텐서를 생성하는 함수입니다.
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15],
                      [0.4, 0.5, 0.1]]])

In [ ]:
# -1은 남은 차원을 자동으로 계산하라는 뜻
#(-1,) 전체는 1차원 텐서로 만들되
# 그 길이는 전체 원소 수에 맞춰 자동 계산하라는 의미
# != -100 -> Boolean 마스크로 만든다 -> 1차원 불리언 텐서
active_loss = tf.reshape(labels,(-1,)) != -100
print(active_loss)


### 📌  기본 개념
#### 1️⃣ `tf.reshape(tensor, new_shape)`

* 텐서의 모양(차원)을 바꾸는 함수야.
* **데이터는 그대로 두고, 배치를 풀거나 붙이기 위해 주로 사용**해.
* 예: `[1, 4, 3]` → `[4, 3]`

#### 2️⃣ `tf.boolean_mask(tensor, mask)`

* `tensor`에서 `mask == True`인 위치만 골라내는 함수야.
* 예:

  ```python
  x = [[10, 20], [30, 40], [50, 60]]
  mask = [True, False, True]
  tf.boolean_mask(x, mask)
  → 결과: [[10, 20], [50, 60]]
  ```



### ✅ 지금 우리가 다루는 데이터

```python
# 정답 라벨
labels = tf.constant([[-100, 2, 1, -100]])  # shape: [1, 4]

# 모델 예측 결과 (logits)
logits = tf.constant([[
    [0.8, 0.1, 0.1],    # 토큰1
    [0.06, 0.04, 0.9],  # 토큰2
    [0.75, 0.1, 0.15],  # 토큰3
    [0.4, 0.5, 0.1]     # 토큰4
]])  # shape: [1, 4, 3]
```



### 👣 한 줄씩 따라가 보자


#### ✅ 1. `tf.reshape(logits, (-1, 3))`

```python
reshaped_logits = tf.reshape(logits, (-1, 3))
```

* 원래 logits는 `[1, 4, 3]` → (배치 1, 4개의 토큰, 클래스 3개)
* reshape 결과는 `[4, 3]`이야.

```python
reshaped_logits = tf.constant([
  [0.8, 0.1, 0.1],     # CLS 토큰
  [0.06, 0.04, 0.9],   # 단어 1
  [0.75, 0.1, 0.15],   # 단어 2
  [0.4, 0.5, 0.1]      # PAD 토큰
])
```

* 즉, 토큰 1개당 클래스 3개 점수를 표현하는 \[4, 3]짜리 배열이 됨


#### ✅ 2. `active_loss = tf.reshape(labels, (-1,)) != -100`

```python
# labels = [[-100, 2, 1, -100]]
active_loss = tf.reshape(labels, (-1,)) != -100
```

* `tf.reshape(labels, (-1,))` → `[ -100, 2, 1, -100 ]` (1차원으로 펴줌)
* `!= -100` → \[False, True, True, False]

이건 뭐냐면:

* 1번째 토큰 (CLS): -100 → 무시할 거니까 False
* 2번째, 3번째 토큰: 라벨 있음 → True
* 4번째 토큰 (PAD): -100 → 무시할 거니까 False

```python
active_loss = [False, True, True, False]
```



#### ✅ 3. `tf.boolean_mask(reshaped_logits, active_loss)`

```python
reduced_logits = tf.boolean_mask(reshaped_logits, active_loss)
```

* `[4, 3]`짜리 reshaped\_logits 중에서,
* `active_loss == True`인 행만 남겨!

즉,

```python
reshaped_logits = [
  [0.8, 0.1, 0.1],     # 무시됨
  [0.06, 0.04, 0.9],   # 포함
  [0.75, 0.1, 0.15],   # 포함
  [0.4, 0.5, 0.1]      # 무시됨
]
active_loss = [False, True, True, False]
```

⇒ 결과:

```python
reduced_logits = [
  [0.06, 0.04, 0.9],
  [0.75, 0.1, 0.15]
]
```


### 🧠 총정리 요약

| 단계                               | 설명                                                   |
| -------------------------------- | ---------------------------------------------------- |
| `reshape(logits, (-1, 3))`       | `[1, 4, 3]` → `[4, 3]`로 펼쳐서 각 토큰마다의 클래스 예측을 1차 정렬    |
| `reshape(labels, (-1,)) != -100` | `[1, 4]` → `[4]`로 펼친 후, 유효한 토큰만 골라내는 boolean mask 생성 |
| `boolean_mask(logits, mask)`     | 유효한 토큰의 예측 결과만 추출 (손실 계산용)                           |


필요하면 `labels` 도 똑같이 `boolean_mask`로 줄여서 loss 계산하면 돼.
그 부분도 이어서 설명해줄 수 있어!
이해됐는지 알려줘 — 궁금한 부분은 더 쉽게 다시 풀어줄게.


In [ ]:
# boolean_mask 함수를 통하여 False 자리의 값 마스킹
reduced_logits = tf.boolean_mask(tf.reshape(logits,(-1, shape_list(logits)[2])), active_loss)
reduced_logits

In [ ]:
labels = tf.boolean_mask(tf.reshape(labels,(-1,)), active_loss)
labels

# 손실 함수의 Reduction이란?

모델이 한 번에 여러 데이터(배치)를 처리할 때, 각 데이터마다 손실값이 나옵니다.

**Reduction**은 이 손실값들을 하나의 값으로 어떻게 줄일지(통합할지) 정하는 옵션이에요.


## 주요 Reduction 옵션

- **NONE**  
  → 각 샘플별 손실을 그대로 유지 (줄이지 않음)  
  → 토큰별 손실을 따로 보고 싶을 때 사용

- **SUM**  
  → 모든 손실값을 다 더함  
  → 배치 전체 손실을 합산

- **SUM_OVER_BATCH_SIZE** (또는 **AUTO**)  
  → 손실값의 평균을 계산 (보통 이게 기본값)  
  → 배치 크기로 나누어 평균 손실 계산


In [ ]:
# 위의 예시 코드들을 바탕으로 레이블 값이 -100 인 경우를 학습 시에 무시하는 방법 적용 -> 손실 함수 구현
def compute_loss(labels, logits):
  # 다중 클래스 분류 문제에서 softmax 사용 x -> from_logits=True 설정
  # NER 같은 다중 클래스 분류에서는 보통 레이블이 정수 인덱스 형태로 존재하기 때문에 -> SparseCategoricalCrossentropy(정수값)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
      # from_logits=True는 입력값이 아직 softmax를 거치지 않은 원시 점수(로짓, logits)임을 알려주는 옵션 -> 손실 함수가 내부적으로 softmax계싼 -> 확률 변환 -> 손실 계산
      # 손실함수 계산시 모든 배치 데이터의 손실 평균 or 합산 x -> 각 데이터별 손실을 개별적으로 반환
      # 개체명 인식(Named Entity Recognition, NER) 문제는 시퀀스 레이블링 문제라서, 입력 문장의 각 토큰마다 클래스를 예측해야 해요. 즉, 토큰별로 손실(loss)을 계산
      # 만일 합쳐지거나 평균내지면 -> 토큰별 세밀한 조정 불가
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE
      )
  # -100 값 반영 x 하도록 라벨 수정
  active_loss = tf.reshape(labels,(-1,))!= -100

  # 라벨과 로짓에 마스킹 적용
  reduced_logits = tf.boolean_mask(tf.reshape(logits,(-1,shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels,(-1,)),active_loss)

  return loss_fn(labels, reduced_logits)

In [ ]:
model = TFBertForSequenceClassification("klue/bert-base",num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(5e-5)
model.compile(optimizer=optimizer, loss=compute_loss)

# 개체명 인식에서 F1 Score를 에폭마다 계산하는 이유

- **정확도(Accuracy)는 부족해요**  
  토큰 하나하나만 맞았는지 보기 때문에, 실제 개체명 전체를 잘 찾았는지 알기 어려움

- **F1 Score가 더 좋아요**  
  - 정밀도(모델이 맞다고 한 것 중 진짜 맞은 비율)와  
  - 재현율(진짜 개체명 중 모델이 찾아낸 비율)을 함께 평가
  둘을 잘 조화시킨 값입니다.

- **개체명 인식 문제에 딱 맞아요**  
  개체명은 시작과 끝, 그리고 라벨까지 정확히 맞아야 하므로,  
  F1 Score가 성능을 정확히 보여줌

- **그래서 매 에폭마다 F1 Score를 계산해요**  
  모델이 점점 더 잘 학습되는지 효과적으로 확인하기 위해서



## 📌 F1score 콜백 클래스 설명

이 코드는 **Keras 모델 학습 중 매 에폭마다 F1 score와 분류 리포트를 출력해주는 콜백** 클래스입니다. 주로 **개체명 인식(NER)** 같은 시퀀스 라벨링 문제에 사용됩니다.



### 🔧 클래스 정의

```python
class F1score(tf.keras.callbacks.Callback):
```

* `tf.keras.callbacks.Callback`을 상속받아 Keras 학습 과정에 끼어들 수 있음
* 매 에폭이 끝날 때 자동으로 평가 로직 실행



### 🏷️ 생성자 (`__init__`)

```python
def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test
```

* 테스트용 입력 (`X_test`)과 정답 라벨 (`y_test`)을 받아 저장
* 이후 에폭마다 모델 성능을 이 데이터로 평가


### 🔁 시퀀스를 텍스트 태그로 변환 (`sequences_to_tags`)

```python
def sequences_to_tags(self, label_ids, pred_ids):
```

* 모델이 출력한 예측값 (`pred_ids`)과 실제 라벨값 (`label_ids`)을 텍스트 태그(예: `B-PER`, `O`, `I-LOC` 등)로 변환

#### 주요 처리:

* `-100`인 라벨은 무시함 (보통 패딩 토큰용)
* `index_to_tag`를 사용해 정수 인덱스를 텍스트 태그로 변환

**반환값**:

* `label_list`: 실제 정답 태그들
* `pred_list`: 모델이 예측한 태그들


### ✅ 에폭 종료 시 실행 (`on_epoch_end`)

```python
def on_epoch_end(self, epoch, logs={}):
```

* 각 에폭이 끝날 때 자동으로 호출됨

#### 수행 순서:

1. **예측 수행**

   ```python
   y_predicted = self.model.predict(self.X_test)
   y_predicted = np.argmax(y_predicted, axis=2)
   ```

   * 모델이 출력한 확률 분포에서 가장 높은 확률을 가지는 클래스 인덱스를 선택

2. **정답 및 예측 태그로 변환**

   ```python
   label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
   ```

3. **F1 점수 계산 및 출력**

   ```python
   score = f1_score(label_list, pred_list, suffix=True)
   print(' - f1: {:04.2f}'.format(score * 100))
   ```

4. **자세한 분류 리포트 출력**

   ```python
   print(classification_report(label_list, pred_list, suffix=True))
   ```


### 🧩 사용 전 확인해야 할 것

* `index_to_tag`: 인덱스를 태그로 매핑하는 딕셔너리 (전역 변수로 정의되어 있어야 함)
* `f1_score`, `classification_report`: `seqeval` 패키지에서 가져온 함수여야 함

```python
from seqeval.metrics import f1_score, classification_report
```


이 콜백은 특히 **NER 모델 학습 중 실시간 평가를 위해 매우 유용**하며, `model.fit()` 시 `callbacks=[F1score(...)]` 형태로 사용됩니다.


In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        # 테스트 데이터(입력, 정답)를 콜백 객체에 저장
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
        # 모델 출력 및 실제 레이블을 텍스트 태그 리스트로 변환하는 함수
        label_list = []  # 실제 레이블 태그를 담을 리스트
        pred_list = []   # 예측된 레이블 태그를 담을 리스트

        # 배치(문장) 단위로 반복
        for i in range(0, len(label_ids)):
            label_tag = []  # 현재 문장 실제 태그 리스트
            pred_tag = []   # 현재 문장 예측 태그 리스트

            # 각 문장 내 토큰별로 실제 레이블과 예측 레이블 비교
            # -100은 패딩 토큰에 해당, 평가에서 제외해야 함
            for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
                if label_index != -100:  # 패딩이 아닌 실제 토큰만 처리
                    # 정수 인덱스를 실제 태그 문자열로 변환
                    label_tag.append(index_to_tag[label_index])
                    pred_tag.append(index_to_tag[pred_index])

            # 변환된 문장 단위 태그 리스트를 전체 리스트에 추가
            label_list.append(label_tag)
            pred_list.append(pred_tag)

        # 실제 레이블과 예측 레이블 리스트 반환
        return label_list, pred_list

    # 매 학습 에폭이 끝날 때 자동으로 호출되는 함수
    def on_epoch_end(self, epoch, logs={}):
        # 테스트 입력(X_test)에 대해 현재 모델의 예측 수행
        y_predicted = self.model.predict(self.X_test)
        # 예측 결과에서 가장 높은 확률을 가진 클래스를 선택 (argmax)
        # axis=2는 각 토큰별 클래스 차원에서 선택한다는 의미 (batch, seq_len, num_classes)
        # argmax	최댓값의 위치/인덱스
        y_predicted = np.argmax(y_predicted, axis=2)

        # 실제 레이블과 예측 레이블을 텍스트 태그 리스트로 변환
        label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

        # F1 점수 계산 (개체명 인식 성능 지표)
        # f1_score 함수:
        #   - 정밀도(Precision): 예측한 개체 중 실제로 맞는 비율
        #   - 재현율(Recall): 실제 개체 중 올바르게 찾아낸 비율
        #   - F1 = 2 × (Precision × Recall) / (Precision + Recall)
        #
        # suffix=True 파라미터:
        #   - BIO 태깅 방식에서 접미사(-PER, -LOC 등)를 기준으로 개체 단위 평가
        #   - 예: ['B-PER', 'I-PER'] → 하나의 PER 개체로 취급
        #   - 개체의 시작과 끝 위치가 완전히 일치해야만 정답으로 인정 (엄격한 평가)
        #   - 부분 일치는 0점 처리 (예: "홍길동" 중 "홍길"만 예측하면 오답)
        score = f1_score(label_list, pred_list, suffix=True)
        # 계산된 F1 점수 출력 (백분율로 보기 좋게 변환)
        print(' - f1: {:04.2f}'.format(score * 100))
        # 각 클래스별 정밀도, 재현율, F1 점수 등 자세한 리포트 출력
        print(classification_report(label_list, pred_list, suffix=True))


In [ ]:
f1_score_report = F1score(X_test,y_test)

model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks=[f1_score_report]
)



## 📌 `convert_examples_to_features_for_prediction` 함수 설명

이 함수는 **예측용 문장 리스트를 BERT 기반 입력 포맷**으로 변환합니다.
NER(Named Entity Recognition) 또는 시퀀스 태깅 모델의 입력으로 사용됩니다.



### 🧩 함수 정의

```python
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                                pad_token_id_for_segment=0,
                                                pad_token_id_for_label=-100):
```

#### 파라미터 설명

| 파라미터                       | 설명                                                              |
| -------------------------- | --------------------------------------------------------------- |
| `examples`                 | 문장 단위의 단어 리스트 (예: `[['Hello', 'world'], ['My', 'name', 'is']]`) |
| `max_seq_len`              | 최대 시퀀스 길이 (예: 128)                                              |
| `tokenizer`                | HuggingFace의 tokenizer 객체                                       |
| `pad_token_id_for_segment` | 세그먼트 ID용 패딩값 (보통 0)                                             |
| `pad_token_id_for_label`   | 라벨용 패딩값 (보통 -100로 마스킹)                                          |



### 🔄 전체 처리 흐름 요약

1. 각 문장을 서브워드 단위로 토크나이징
2. 서브워드 처리 시 **첫 서브워드만 라벨을 예측**하고, 나머지는 무시(`-100`)
3. `[CLS]`와 `[SEP]` 토큰 추가 (레이블도 맞춰서 `-100` 추가)
4. 정수 인코딩 + attention mask + segment ID 생성
5. 필요한 경우 길이를 `max_seq_len`으로 패딩 처리
6. 모두 `numpy array`로 반환



### 🧱 내부 주요 단계 설명

#### 1. 서브워드 토크나이징 & 라벨 마스킹

```python
subword_tokens = tokenizer.tokenize(one_word)
tokens.extend(subword_tokens)
label_mask.extend([0] + [pad_token_id_for_label] * (len(subword_tokens) - 1))
```

* 예: `playing → ['play', '##ing']` 이라면
  `label_mask = [0, -100]`
  (첫 서브워드만 레이블 평가 대상)


#### 2. `[CLS]`와 `[SEP]` 토큰 추가

```python
tokens = [cls_token] + tokens + [sep_token]
label_mask = [pad_token_id_for_label] + label_mask + [pad_token_id_for_label]
```

* 입력 앞뒤에 특수 토큰 추가
* 라벨 마스크에도 맞춰 `-100` 삽입


#### 3. 정수 인코딩 및 패딩

```python
input_id = tokenizer.convert_tokens_to_ids(tokens)
padding_count = max_seq_len - len(input_id)

input_id += [pad_token_id] * padding_count
attention_mask = [1] * len(tokens) + [0] * padding_count
token_type_id = [pad_token_id_for_segment] * max_seq_len
label_mask += [pad_token_id_for_label] * padding_count
```

* 토큰을 정수로 변환
* 부족한 길이만큼 패딩 추가
* attention mask는 실제 토큰에는 1, 패딩에는 0



#### 4. 길이 검증

```python
assert len(input_id) == max_seq_len
assert len(attention_mask) == max_seq_len
...
```

* 전처리가 정확히 작동했는지 확인



### 📤 반환값

```python
return (input_ids, attention_masks, token_type_ids), label_masks
```

* `input_ids`: BERT 입력용 정수 토큰 배열
* `attention_masks`: 1은 실제 토큰, 0은 패딩
* `token_type_ids`: 문장 구분용 세그먼트 (보통 NER에서는 모두 0)
* `label_masks`: 실제 평가할 토큰은 0, 나머지는 `-100` (loss 계산 제외용)



### ✅ 최종 사용 예시

```python
X = [['오늘', '날씨', '좋다'], ['나는', '학생입니다']]
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

inputs, label_masks = convert_examples_to_features_for_prediction(X, 64, tokenizer)
```

* 예측 또는 추론용 BERT 모델 입력 생성에 유용
* 라벨은 없지만 라벨 마스크는 평가용 필터링을 위해 유지됨




In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                             pad_token_id_for_segment=0,
                                             pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            # 하나의 단어에 대해서 서브워드로 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            # 서브워드로 쪼개진 첫번째 서브워드를 제외하고 그 뒤의 서브워드들은 -100으로 마
            # 스킹함
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경
        # 우 max_seq_len - 2의 길이로 변환.
        # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이
        # 62로 변환.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        # [SEP]를 추가하는 코드
        # 1. 토르화 결과의 맨 뒷 부분에 [SEP] 토큰 추가
        # 2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        # [CLS]를 추가하는 코드
        # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
        # 2. 레이블의 맨 앞 부분에도 -100 추가.
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)

        # 어텐션 마스크 생성
        attention_mask = [1] * len(input_id)

        # 정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(input_id)

        # 정수 인코딩, 어텐션 마스크에 패딩 추가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)

        # 세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment] * max_seq_len

        # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.array(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(
    test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

print('기존 원문 :', test_data_sentence[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레이블 마스크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])


## 📌 `ner_prediction` 함수 설명

이 함수는 문장 리스트에 대해 **NER(Named Entity Recognition)** 예측 결과를 반환합니다.
예측 결과는 각 단어에 대해 (단어, 예측된 개체명 태그) 형태로 구성됩니다.



### 🧩 함수 정의

```python
def ner_prediction(examples, max_seq_len, tokenizer):
```

#### 파라미터 설명

| 파라미터          | 설명                                          |
| ------------- | ------------------------------------------- |
| `examples`    | 문자열 문장 리스트 (예: `["나는 학생입니다", "이순신은 장군이다"]`) |
| `max_seq_len` | 모델 입력의 최대 길이 (예: 128)                       |
| `tokenizer`   | HuggingFace의 BERT tokenizer 객체              |



### 🔄 전체 처리 흐름

1. 문장들을 단어 리스트로 변환
2. 예측용 입력 포맷으로 변환 (`convert_examples_to_features_for_prediction` 호출)
3. 모델 예측 수행
4. 예측된 라벨 인덱스를 텍스트 태그로 변환
5. (단어, 예측 태그) 형태로 결과 구성



### 1️⃣ 문장 분할 (단어 리스트로 변환)

```python
examples = [sent.split() for sent in examples]
```

* 문자열 문장 → 단어 단위 리스트로 변환
* 예: `"나는 학생입니다"` → `['나는', '학생입니다']`



### 2️⃣ 입력 포맷 변환

```python
X_pred, label_masks = convert_examples_to_features_for_prediction(...)
```

* 문장을 BERT 입력 형식으로 변환
* 반환값:

  * `X_pred`: (input\_ids, attention\_masks, token\_type\_ids)
  * `label_masks`: 실제 평가 대상인 토큰만 `0`, 나머지는 `-100`



### 3️⃣ 모델 예측 수행

```python
y_predicted = model.predict(X_pred)
y_predicted = np.argmax(y_predicted, axis=2)
```

* 모델 출력: (배치, 시퀀스 길이, 클래스 수)
* 가장 높은 확률을 갖는 클래스 인덱스 선택



### 4️⃣ 예측값을 태그로 디코딩

```python
for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
    if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])
```

* `-100`인 위치는 무시 (예: `[CLS]`, `[SEP]`, 서브워드 등)
* 나머지 위치의 예측 인덱스를 태그로 변환 (예: `3 → B-PER`)



### 5️⃣ 단어와 예측 태그를 묶기

```python
for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
        one_sample_result.append((one_word, label_token))
```

* 예:
  입력: `['이순신', '장군']`
  예측: `['B-PER', 'O']`
  결과: `[('이순신', 'B-PER'), ('장군', 'O')]`



### 📤 반환값

```python
return result_list
```

* 결과 형태: `[[('단어1', '태그1'), ('단어2', '태그2'), ...], [...], ...]`
* 문장 단위로 단어와 태그가 묶여 있음


### ✅ 예시 사용

```python
sentences = ["이순신은 장군이다", "삼성은 한국 기업이다"]
results = ner_prediction(sentences, 128, tokenizer)

# 출력 예시
# [[('이순신은', 'B-PER'), ('장군이다', 'O')],
#  [('삼성은', 'B-ORG'), ('한국', 'B-LOC'), ('기업이다', 'O')]]
```


### 🔔 주의사항

* 전역 변수 `model`, `index_to_tag` 가 정의되어 있어야 정상 작동합니다.

  ```python
  # 예시
  model = tf.keras.models.load_model(...)
  index_to_tag = {0: 'O', 1: 'B-PER', 2: 'I-PER', ...}
  ```

* 입력 문장은 띄어쓰기 기준으로 나뉘므로 형태소 기준으로 나누지 않는 한 성능에 영향을 줄 수 있음


In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):
    examples = [sent.split() for sent in examples]
    X_pred, label_masks = convert_examples_to_features_for_prediction(
        examples,
        max_seq_len=128,
        tokenizer=tokenizer
    )
    y_predicted = model.predict(X_pred)
    # 마지막 차원 클래스차원에서 최대값을 찾는다
    # 마지막 차원 = 클래스 차원 -> 각 클래스 확률이 나와있을거니 아마도 softmax -> 그중에 최대값 -> 해당 클래스일 확률 높음
    y_predicted = np.argmax(y_predicted, axis=2)

    pred_list = []
    result_list = []

    for i in range(0, len(label_masks)):
        pred_tag = []

        # ex) 모델의 예측값 디코딩 과정
        # 예측값(y_predicted)에서 레이블 마스크(label_masks)의 값이 -100인 동일 위치의 값은 삭제
        # label_masks : [-100 -100 0 0 -100]
        # y_predicted : [ 0   2   0 2   0 ] ==> [1 2] ==> 최종 예측(pred_tag) : [PER-B PER-I]
        for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
            if label_index != -100:
                pred_tag.append(index_to_tag[pred_index])

        pred_list.append(pred_tag)

    for example, pred in zip(examples, pred_list):
        one_sample_result = []
        for one_word, label_token in zip(example, pred):
            one_sample_result.append((one_word, label_token))
        result_list.append(one_sample_result)

    return result_list




## 차원 변화 과정

### 1. 원본 `y_predicted`

- 모델 출력: `(batch_size, sequence_length, num_classes)`
- 예시: `y_predicted.shape = (2, 5, 9)`  
  (배치 2개, 시퀀스 길이 5, 클래스 수 9)

```python
y_predicted = [
    # 첫 번째 문장
    [
        [0.1, 0.0, 0.9, 0.0, ...],  # 첫 번째 토큰의 각 클래스 확률 -> 2
        [0.8, 0.1, 0.1, 0.0, ...],  # 두 번째 토큰의 각 클래스 확률 -> 0
        [0.0, 0.0, 0.0, 0.9, ...],  # 세 번째 토큰의 각 클래스 확률 -> 3
        ...
    ],
    # 두 번째 문장
    [
        ...
    ]
]
````



### 2. `argmax` 적용 후

```python
y_predicted = np.argmax(y_predicted, axis=2)
```

* 결과 차원: `(batch_size, sequence_length)` → 여전히 2차원!
* `y_predicted.shape = (2, 5)`

```python
y_predicted = [
    [2, 0, 3, 1, 0],  # 첫 번째 문장의 각 토큰 예측 클래스
    [1, 2, 0, 0, 4]   # 두 번째 문장의 각 토큰 예측 클래스
]
```



In [ ]:
# 예측할 문장
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

# 문장 리스트
test_samples = [sent1, sent2]

# NER 예측 함수 호출 (128 토큰 제한)
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)

# 결과 출력
result_list